<a href="https://colab.research.google.com/github/Bhandari007/sequence_model_course/blob/main/Dinosaurus_Island_Character_Level_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week1/w1a2/dinos.txt
! wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week1/w1a2/generateTestCases.py
! wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week1/w1a2/shakespeare.txt
! wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week1/w1a2/shakespeare_utils.py
! wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week1/w1a2/test_utils.py
! wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week1/w1a2/utils.py

--2022-10-17 14:41:33--  https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week1/w1a2/dinos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19909 (19K) [text/plain]
Saving to: ‘dinos.txt’

dinos.txt           100%[===================>]  19.44K  --.-KB/s    in 0s      

2022-10-17 14:41:33 (82.0 MB/s) - ‘dinos.txt’ saved [19909/19909]

--2022-10-17 14:41:34--  https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week1/w1a2/generateTestCases.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:4